In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Sir Cocoa of Butter\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
#Visit Site
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
# Parse results
html = browser.html
soup = bs(html, 'html.parser')

#Search for news titles
news_titles = soup.find_all('div', class_='content_title')

#Search for paragraph text
paragraph_text = soup.find_all('div', class_='article_teaser_body')

#Extract first title and paragraph, and assign to variables
first_titles = news_titles[0].text
ftitle_p_text = paragraph_text[0].text

print(first_titles)
print(ftitle_p_text)


Mars Now
NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.


In [5]:
#NASA JPL
browser.visit("https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html")

In [6]:
#Use splinter to navigate to image

browser.links.find_by_partial_text('FULL IMAGE')




In [7]:
html = browser.html
image_soup = bs(html, 'html.parser')

#Search for image source
featured_image = image_soup.find_all('img', class_='headerimage')
relative_image_path = featured_image[0].a['href']
featured_image_url = 'https://www.jpl.nasa.gov' + relative_image_path

print(featured_image_url)

TypeError: 'NoneType' object is not subscriptable

In [8]:
#Use Pandas to scrape data

mars_facts = pd.read_html('https://space-facts.com/mars/')[0]

print(mars_facts)

mars_facts.columns=["Description", "Value"]
mars_facts.set_index("Description", inplace=True)
mars_facts

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
mars_facts_html = [mars_facts.to_html(classes='data table table-borderless', index=False, header=False, border=0)]
mars_facts_html

['<table border="0" class="dataframe data table table-borderless">\n  <tbody>\n    <tr>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>']

In [11]:
# Open browser to USGS Astrogeology site
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

In [12]:
html = browser.html
soup = bs(html, 'html.parser')

hemi_names = []

# Search for the names of all four hemispheres
soup_results = soup.find_all('div', class_="collapsible results")
hemispheres = soup_results[0].find_all('h3')

# Get text and store in list
for name in hemispheres:
    hemi_names.append(name.text)

hemi_names

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [15]:
#Search for image links

image_search_results = soup_results[0].find_all('a')
image_links = []

for image in image_search_results:
    if(image.img) :
        image_url = 'https://astrogeology.usgs.gov/' + image['href']
        image_links.append(image_url)

image_links


['https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced']

In [17]:
# Zip together the list of hemisphere names and hemisphere image links
mars_hemi_zip = zip(hemi_names, image_links)

hemisphere_image_urls = []

# Iterate through the zipped object
for title, img in mars_hemi_zip:
    
    mars_hemi_dict = {}
    
    # Add hemisphere title to dictionary
    mars_hemi_dict['title'] = title
    
    # Add image url to dictionary
    mars_hemi_dict['img_url'] = img
    
    # Append the list with dictionaries
    hemisphere_image_urls.append(mars_hemi_dict)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced'}]